# Agregación y agrupación de datos 

## Agregación de datos categóricos 

Es posible agregar datos categóricos a un Data Frame de manera aleatoria. Al definir un conjunto de valores, la función `choice ()` selecciona de manera aleatoria un elemento de este conjunto.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
gender = ["Male", "Female"]
income = ["Poor", "Middle Class", "Rich"]

In [ ]:
n = 500
gender_data = []
income_data = []

for i in range(0, 500):
    gender_data.append(np.random.choice(gender))
    income_data.append(np.random.choice(income))

In [ ]:
gender_data[:10]

In [ ]:
income_data[:10]

In [ ]:
height = 160 + 30 * np.random.randn(n)
weight = 65 + 25 * np.random.randn(n)
age = 30 + 12 * np.random.randn(n)
income = 18000 + 3500 * np.random.randn(n)

In [ ]:
data = pd.DataFrame(
    {
        "Gender": gender_data,
        "Economic Status": income_data,
        "Height": height,
        "Weight": weight,
        "Age": age,
        "Income": income
    }
)

In [ ]:
data.head()

## Agrupación de datos

In [ ]:
grouped_gender = data.groupby("Gender")

In [ ]:
grouped_gender.groups

In [ ]:
for name, group in grouped_gender:
    print(name)
    print(group)

In [ ]:
grouped_gender.get_group("Female")

In [ ]:
double_group = data.groupby(["Gender", "Economic Status"])

In [ ]:
for name, group in double_group:
    print(name)
    print(group)

### Operaciones sobre datos agrupados

In [ ]:
double_group.sum()

In [ ]:
double_group.mean()

In [ ]:
double_group.size()

In [ ]:
double_group.describe()

In [ ]:
grouped_gender["Age"].describe()

In [ ]:
double_group.aggregate(
    {
        "Income": np.sum,
        "Age": np.mean,
        "Height": np.std
    }
)

Con el uso de funciones `lambda` se pueden hacer agregaciones de datos resultado de una operación particular. 

In [ ]:
double_group.aggregate(
    {
        "Age": np.mean,
        "Height": lambda h:(np.mean(h))/np.std(h)
    }
)

In [ ]:
double_group.aggregate([np.sum, np.mean, np.std])

In [ ]:
double_group.aggregate([lambda x:np.mean(x)/np.std(x)])

### Filtrado de datos

In [ ]:
double_group.sum()

In [ ]:
# devuelve los elementos "Age" que agrupados
# sumaron mas de 2400
double_group["Age"].filter(lambda x: x.sum() > 2400)

### Transformación de variables

In [ ]:
# define la transformación de datos mediante una función lambda
zscore = lambda x : (x - x.mean())/x.std()

In [ ]:
z_group = double_group.transform(zscore)

In [ ]:
plt.hist(z_group["Age"])

In [ ]:
# define una lambda que modifica valores nulos con la media de una columna
fill_mean = lambda x: x.fillna(x.mean())

In [ ]:
double_group.transform(fill_mean)

### Operaciones diversas muy útiles

In [ ]:
# primera ocurrencia de datos agrupados
double_group.head(1)

In [ ]:
# última ocurrencia de datos agrupados
double_group.tail(1)

In [ ]:
# n-ésima ocurrencia de datos agrupados
double_group.nth(32)

In [ ]:
# ordenación de un dataframe por valores en una o más columnas
data_sorted = data.sort_values(["Age", "Income"])

In [ ]:
data_sorted.head(10)

In [ ]:
# agrupa un dataframe ordenado
age_grouped = data_sorted.groupby("Gender")

In [ ]:
# muestra las dos primeras ocurrencias ordenadas de cada grupo 
age_grouped.head(2)

In [ ]:
# muestra las dos últimas ocurrencias ordenadas de cada grupo
age_grouped.tail(2)

## Conjunto de entrenamiento y conjunto de prueba

In [ ]:
mainpath = "/home/oscar/Escritorio/misnotebooks/data/"
filename = "Customer Churn Model.txt"
data = pd.read_csv(mainpath + filename)
data.head()

In [ ]:
len(data)

### Dividir utilizando la distribución normal

In [ ]:
a = np.random.randn(len(data))
type(a)

In [ ]:
plt.hist(a)

In [ ]:
check = (a < 0.8)
check

**Nota1:** Se intentó hacer un histograma del array `check` que contiene valores booleanos después aplicar el filtrado con el objetivo de mostrar como se han subdividido los datos. No obstante, en Python versión 3.6.5 con Matplotlib versión 2.2.2 el histograma no se realiza sin la conversión de tipo booleano a entero. 

**Nota2:** Para resolver lo anterior revisé la sección https://www.python-course.eu/numpy_masking del curso *Numerical Python Course* que recomiendo seguir estudiando.  

In [ ]:
# https://www.python-course.eu/numpy_masking.php
check_int = check.astype(np.int)

In [ ]:
plt.hist(check_int)

Se han filtrado el 80% de los datos, separados del otro 20%, tales datos subdivididos pueden asignarse a dos diferentes subgrupos: *training y testing*

In [ ]:
training = data[check]
testing = data[~check]

In [ ]:
len(training)

In [ ]:
len(testing)

### Dividir utilizando Sci-kit Learn

**Nota:** Anteriormente se utilizaba la función `train_test_split()` perteneciente al módulo `cross_validation` el cual caduduca en la versión 19.02 de Sci-kit Learn, en su lugar se utiliza el módulo `model_selection` para versiones posteriores del Sci-kit Learn.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(data, test_size = 0.2)

In [ ]:
len(train)

In [ ]:
len(test)

### Usando una función de shuffle

In [ ]:
data.head()

In [ ]:
import sklearn

In [ ]:
data_shuffle = sklearn.utils.shuffle(data)

In [ ]:
data_shuffle.head()

In [ ]:
cut = int(0.80 * len(data_shuffle))
train_data = data_shuffle[: cut]
test_data = data_shuffle[cut + 1 :]

In [ ]:
len(train_data)

In [ ]:
len(test_data)